In [ ]:
# !pip uninstall fsspec -y
# !pip install --upgrade fsspec
# !pip install transformers accelerate datasets

In [1]:
!nvidia-smi

Sat Jul 17 06:19:24 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount('/content/drive')

!pip install -q kaggle
!mkdir /root/.kaggle
!cp /content/drive/MyDrive/Colab\ Notebooks/kaggle.json /root/.kaggle/

Mounted at /content/drive


In [5]:
!mkdir /content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/roberta-base-nishipy
# !unzip -n archive(7)  -d /content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/clrp-roberta-base

mkdir: cannot create directory ‘/content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/roberta-base-nishipy’: File exists


In [10]:
!unzip -n /content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/roberta_nishipy.zip -d /content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/roberta-base-nishipy

Archive:  /content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/roberta_nishipy.zip
  inflating: /content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/clrp-roberta-base/model_1.pth  
  inflating: /content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/clrp-roberta-base/model_2.pth  
  inflating: /content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/clrp-roberta-base/model_3.pth  
  inflating: /content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/clrp-roberta-base/model_4.pth  
  inflating: /content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/clrp-roberta-base/model_5.pth  


In [11]:
!pip install transformers datasets accelerate
!pip install transformers
!pip install colorama

     |████████████████████████████████| 2.5MB 7.1MB/s 
     |████████████████████████████████| 266kB 56.0MB/s 
     |████████████████████████████████| 51kB 7.5MB/s 
     |████████████████████████████████| 901kB 45.5MB/s 
     |████████████████████████████████| 3.3MB 49.5MB/s 
     |████████████████████████████████| 245kB 55.2MB/s 
     |████████████████████████████████| 122kB 52.1MB/s 


# Define

In [12]:
import os
import math
import random
import time

import numpy as np
import pandas as pd


import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from transformers import AdamW
from transformers import AutoTokenizer
from transformers import AutoModel
from transformers import AutoConfig
from transformers import get_cosine_schedule_with_warmup

from sklearn.model_selection import KFold

import gc
gc.enable()

In [13]:
test_df = pd.read_csv("/content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/test.csv")
submission_df = pd.read_csv("/content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/sample_submission.csv")

In [14]:
NUM_FOLDS = 5
NUM_EPOCHS = 3
BATCH_SIZE = 16
MAX_LEN = 248
EVAL_SCHEDULE = [(0.50, 16), (0.49, 8), (0.48, 4), (0.47, 2), (-1., 1)]
ROBERTA_PATH = "/content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/huggingface-roberta-variants/roberta-base/"
TOKENIZER_PATH = "/content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/huggingface-roberta-variants/roberta-base/"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [15]:
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH)

In [16]:
class LitDataset(Dataset):
    def __init__(self, df, inference_only=False):
        super().__init__()

        self.df = df        
        self.inference_only = inference_only
        self.text = df.excerpt.tolist()
        #self.text = [text.replace("\n", " ") for text in self.text]
        
        if not self.inference_only:
            self.target = torch.tensor(df.target.values, dtype=torch.float32)        
    
        self.encoded = tokenizer.batch_encode_plus(
            self.text,
            padding = 'max_length',            
            max_length = MAX_LEN,
            truncation = True,
            return_attention_mask=True
        )        
 

    def __len__(self):
        return len(self.df)

    
    def __getitem__(self, index):        
        input_ids = torch.tensor(self.encoded['input_ids'][index])
        attention_mask = torch.tensor(self.encoded['attention_mask'][index])
        
        if self.inference_only:
            return (input_ids, attention_mask)            
        else:
            target = self.target[index]
            return (input_ids, attention_mask, target)

In [17]:
class LitModel(nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(ROBERTA_PATH)
        config.update({"output_hidden_states":True, 
                       "hidden_dropout_prob": 0.0,
                       "layer_norm_eps": 1e-7})                       
        
        self.roberta = AutoModel.from_pretrained(ROBERTA_PATH, config=config)  
            
        self.attention = nn.Sequential(            
            nn.Linear(768, 512),            
            nn.Tanh(),                       
            nn.Linear(512, 1),
            nn.Softmax(dim=1)
        )        

        self.regressor = nn.Sequential(                        
            nn.Linear(768, 1)                        
        )
        

    def forward(self, input_ids, attention_mask):
        roberta_output = self.roberta(input_ids=input_ids,
                                      attention_mask=attention_mask)        

        # There are a total of 13 layers of hidden states.
        # 1 for the embedding layer, and 12 for the 12 Roberta layers.
        # We take the hidden states from the last Roberta layer.
        last_layer_hidden_states = roberta_output.hidden_states[-1]

        # The number of cells is MAX_LEN.
        # The size of the hidden state of each cell is 768 (for roberta-base).
        # In order to condense hidden states of all cells to a context vector,
        # we compute a weighted average of the hidden states of all cells.
        # We compute the weight of each cell, using the attention neural network.
        weights = self.attention(last_layer_hidden_states)
                
        # weights.shape is BATCH_SIZE x MAX_LEN x 1
        # last_layer_hidden_states.shape is BATCH_SIZE x MAX_LEN x 768        
        # Now we compute context_vector as the weighted average.
        # context_vector.shape is BATCH_SIZE x 768
        context_vector = torch.sum(weights * last_layer_hidden_states, dim=1)        
        
        # Now we reduce the context vector to the prediction score.
        
        return last_layer_hidden_states
#         return self.regressor(context_vector)

In [18]:
class CFG:
    model_name_or_path = '/content/drive/MyDrive/CommonLit/input/roberta-transformers-pytorch/roberta-base'
    #model_name_or_path = '../input/huggingface-bert-variants/bert-base-cased/bert-base-cased/'
    clip_by_train_range = False
    batch_size = 8
    max_seq_length = 512
    seq_length = 100
    learning_rate = 2e-5
    use_lr_scheduler = True
    mid_eval = True
    mid_eval_step_num = 50
    random_seed = 2021
    model_output_dir = './'
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
cfg = CFG()
seed_everything=(cfg.random_seed)
QUICK_CHECK = True

global_start_t = time.time()
print('ok')

ok


In [19]:
from sklearn.model_selection import KFold,StratifiedKFold


# def get_embeddings(df, model_path=cfg.model_name_or_path):
#     print('get into get_embeddings()')
#     global model, tokenizer, cfg, collate_fn_test
    
#     model = LitModel(cfg.model_name_or_path)
#     model.to(cfg.device)
#     model.load_state_dict(torch.load(model_path))
#     model.eval()
    
#     ds = LitDataset(df, tokenizer, cfg.max_seq_length)
#     dl = DataLoader(ds, batch_size=2*cfg.batch_size, shuffle=False, 
#                     collate_fn=collate_fn_test, num_workers=0)
# #     (test_dataset, batch_size=BATCH_SIZE,
# #                          drop_last=False, shuffle=False, num_workers=2)


#     embeddings = []
#     with torch.no_grad():
#         for i, inputs in enumerate(dl):
#             inputs = {key:val.reshape(val.shape[0],-1).to(cfg.device) for key,val in inputs.items()}
#             outputs = model.bert(**inputs, return_dict=True)
#             outputs = outputs['pooler_output'].detach().cpu().numpy()
#             embeddings.extend(outputs)
#     del model
#     return np.array(embeddings)


def get_embeddings(df,path,plot_losses=True, verbose=True):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"{device} is used")
            
    model = LitModel()
    model.load_state_dict(torch.load(path))
    model.to(device)
    model.eval()
    
    tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/CommonLit/input/roberta-base')
    
#     ds = LitDataset(df, tokenizer, cfg.max_seq_length)
    ds = LitDataset(df, inference_only=True)
    dl = DataLoader(ds,
                  batch_size = cfg.batch_size,
                  shuffle=False,
                  num_workers = 4,
                  pin_memory=True,
                  drop_last=False
                 )
        
    embeddings = list()
    with torch.no_grad():
        for i, inputs in enumerate(dl):
            print(inputs)
            inputs = {key:val.reshape(val.shape[0],-1).to(device) for key,val in inputs.items()}
            outputs = model(**inputs)
            outputs = outputs.detach().cpu().numpy()
            embeddings.extend(outputs)
    return np.array(embeddings)



# def get_preds_svm(X, y, X_test, bins=bins, nfolds=5, C=10, kernel='rbf', gamma='auto'):
def get_preds_svm(X, y, X_test, nfolds=5, C=10, kernel='rbf', gamma='auto'):
    print('*****'*10 + ' in get_preds_svm() ' + '*****'*10)
    print(f'C: {C}, kernel: {kernel}, gamma: {gamma}')
    scores = []
    preds = np.zeros((X_test.shape[0]))
    
    
    SEED = 1000
    kfold = KFold(n_splits=NUM_FOLDS, random_state=SEED, shuffle=True)
#     kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    for k, (train_idx, valid_idx) in enumerate(kfold.split(X)):
        model = SVR(C=C, kernel=kernel, gamma=gamma)
        X_train, y_train = X[train_idx], y[train_idx]
        X_valid, y_valid = X[valid_idx], y[valid_idx]
        
        model.fit(X_train, y_train)
        prediction = model.predict(X_valid)
        score = rmse_score(prediction, y_valid)
        print(f'Fold {k}, rmse_score: {score:.7f}')
        scores.append(score)
        preds += model.predict(X_test)
        
    mean_valid_rmse = np.mean(scores)
    params = {'C': C, 'kernel': kernel, 'gamma': gamma}
    print(f'mean_valid_rmse: {mean_valid_rmse:.7f}')
    print('*****'*22)
    return mean_valid_rmse, np.array(preds) / nfolds, params

print('ok')

ok


In [21]:
import tqdm

train_df = pd.read_csv('/content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/train.csv')

train_embeddings1 = get_embeddings(train_df, '/content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/roberta-base-nishipy/model_1.pth')
test_embeddings1 = get_embeddings(test_df, '/content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/roberta-base-nishipy/model_1.pth')
print(f'train_embeddings1.shape: {train_embeddings1.shape}  test_embeddings1.shape: {test_embeddings1.shape}')

train_embeddings2 = get_embeddings(train_df, '/content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/roberta-base-nishipy/model_2.pth')
test_embeddings2 = get_embeddings(test_df, '/content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/roberta-base-nishipy/model_2.pth')
print(f'train_embeddings2.shape: {train_embeddings2.shape}  test_embeddings2.shape: {test_embeddings2.shape}')

train_embeddings3 = get_embeddings(train_df, '/content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/roberta-base-nishipy/model_3.pth')
test_embeddings3 = get_embeddings(test_df, '/content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/roberta-base-nishipy/model_3.pth')
print(f'train_embeddings3.shape: {train_embeddings3.shape}  test_embeddings3.shape: {test_embeddings3.shape}')

train_embeddings4 = get_embeddings(train_df, '/content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/roberta-base-nishipy/model_4.pth')
test_embeddings4 = get_embeddings(test_df, '/content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/roberta-base-nishipy/model_4.pth')
print(f'train_embeddings4.shape: {train_embeddings4.shape}  test_embeddings4.shape: {test_embeddings4.shape}')

train_embeddings5 = get_embeddings(train_df, '/content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/roberta-base-nishipy/model_5.pth')
test_embeddings5 = get_embeddings(test_df, '/content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/roberta-base-nishipy/model_5.pth')
print(f'train_embeddings5.shape: {train_embeddings5.shape}  test_embeddings5.shape: {test_embeddings5.shape}')

cuda is used


Some weights of the model checkpoint at /content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/huggingface-roberta-variants/roberta-base/ were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
404 Client Error: Not Found for url: https://huggingface.co//content/drive/MyDrive/CommonLit/input/roberta-base/resolve/main/config.json


OSError: ignored

In [ ]:
def predict(model, data_loader):
    """Returns an np.array with predictions of the |model| on |data_loader|"""
    model.eval()

    result = np.zeros(len(data_loader.dataset))    
    index = 0
    
    with torch.no_grad():
        for batch_num, (input_ids, attention_mask) in enumerate(data_loader):
            input_ids = input_ids.to(DEVICE)
            attention_mask = attention_mask.to(DEVICE)
                        
            pred = model(input_ids, attention_mask)                        

            result[index : index + pred.shape[0]] = pred.flatten().to("cpu")
            index += pred.shape[0]

    return result

# Predict and submit

In [ ]:
all_predictions = np.zeros((5, len(test_df)))

test_dataset = LitDataset(test_df, inference_only=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                         drop_last=False, shuffle=False, num_workers=2)

for index in range(5):
    #CHANGEME
    model_path = f"/content/drive/MyDrive/CommonLit/input/roberta-base-20210711202147-sche/model_{index + 1}.pth"
    print(f"\nUsing {model_path}")
                        
    model = LitModel()
    model.load_state_dict(torch.load(model_path))    
    model.to(DEVICE)
    
    all_predictions[index] = predict(model, test_loader)
    
    del model
    gc.collect()

In [ ]:
predictions = all_predictions.mean(axis=0)
submission_df.target = predictions
print(submission_df)
submission_df.to_csv("submission.csv", index=False)